In [1]:
# import all necessary packages
import pandas as pd

In [2]:
# define the path of the dataset
mnist_path = "/usr/scratch/badile31/msc22f11/msc22f11/PlayGround/mnist_train.csv"
# read it in as a pandas dataframe
mnist_train_df = pd.read_csv(mnist_path)
mnist_train_df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# define how many rows to read in
n_rows = 20
# read in the first n_rows rows of the dataframe
mnist_train_n_rows_df = mnist_train_df[:n_rows]
mnist_train_n_rows_df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# separate the dataframe into the features and the labels
mnist_train_n_rows_features_df = mnist_train_n_rows_df.drop(["label"], axis=1)
mnist_train_n_rows_labels_df = mnist_train_n_rows_df["label"]

In [5]:
# scale the feature values to be between 0 and 1
mnist_train_n_rows_features_scaled_df = mnist_train_n_rows_features_df.div(255)
mnist_train_n_rows_features_scaled_df.head()

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# get the data type of the features
feature_type = str(type(mnist_train_n_rows_features_scaled_df.iloc[0][0])).split(".")[-1].split("'")[0]
print("Data type of the features: {}".format(feature_type))
# get the data type of the labels
label_type = str(type(int(mnist_train_n_rows_labels_df.iloc[0]))).split("<")[-1].split(">")[0].split("class")[-1].replace("'", "").strip()
print("Data type of the labels: {}".format(label_type))

Data type of the features: float64
Data type of the labels: int


In [14]:
# write the feature data into a binary file
for row in mnist_train_n_rows_features_scaled_df.values:
    bin_row = bytearray(row)
    f = open(str(n_rows) + '_features_' + str(feature_type), 'ab')
    f.write(bin_row)
    f.close()
print("Wrote {} rows of features to binary file: {}".format(n_rows, str(n_rows) + '_features_' + str(feature_type)))

Wrote 20 rows of features to binary file: 20_features_float64


In [51]:
# write the label data into a binary file
for row in mnist_train_n_rows_labels_df.values:
    bin_row = int(row).to_bytes(8, byteorder='little')
    f = open(str(n_rows) + '_labels_' + str(label_type), 'ab')
    f.write(bin_row)
    f.close()
print("Wrote {} rows of labels to binary file: {}".format(n_rows, str(n_rows) + '_labels_' + str(label_type)))

Wrote 20 rows of labels to binary file: 20_labels_int


In [30]:
# try alternative file mode for writing binary files for labels
for row in mnist_train_n_rows_labels_df[:n_rows]:
    bin_row = int(row).to_bytes(8, byteorder='little')
    # print the binary representation of the label as 32 bit number
    print(format(int.from_bytes(bin_row, byteorder='little'), '032b'))
    f = open('test_labels_bin8', 'ab')
    f.write(bin_row)
    f.close()

00000000000000000000000000000101
00000000000000000000000000000000
00000000000000000000000000000100
00000000000000000000000000000001
00000000000000000000000000001001
00000000000000000000000000000010
00000000000000000000000000000001
00000000000000000000000000000011
00000000000000000000000000000001
00000000000000000000000000000100
00000000000000000000000000000011
00000000000000000000000000000101
00000000000000000000000000000011
00000000000000000000000000000110
00000000000000000000000000000001
00000000000000000000000000000111
00000000000000000000000000000010
00000000000000000000000000001000
00000000000000000000000000000110
00000000000000000000000000001001
